In [1]:
%%time 
import random
import uproot
import numpy as np

CPU times: user 1.08 s, sys: 4.9 s, total: 5.98 s
Wall time: 1.96 s


In [2]:
#Print all possible Keys
#Thats the Input Data

#List with directions to data files
Data_List = 'Data/Sexa_CustomV0.list'

with open(Data_List) as f:
    input_data = f.read().splitlines()

    with uproot.open(input_data[0]) as file:
        print("Possible Keys:")
        print(file['Sexaquarks'].keys())


Possible Keys:
['RunNumber', 'DirNumber', 'Event', 'Idx_V0A', 'Idx_V0B', 'E', 'Px', 'Py', 'Pz', 'M', 'X', 'Y', 'Z', 'DCA', 'isSignal', 'V0A_Idx_Pos', 'V0A_Idx_Neg', 'V0A_Px', 'V0A_Py', 'V0A_Pz', 'V0A_X', 'V0A_Y', 'V0A_Z', 'V0A_Pos_Px', 'V0A_Pos_Py', 'V0A_Pos_Pz', 'V0A_Neg_Px', 'V0A_Neg_Py', 'V0A_Neg_Pz', 'V0A_isSignal', 'V0A_E_asK0', 'V0A_E_asAL', 'V0A_couldBeK0', 'V0A_couldBeAL', 'V0A_Chi2', 'V0A_DCA_Daughters', 'V0A_IP_wrtPV', 'V0A_CPA_wrtPV', 'V0A_ArmAlpha', 'V0A_ArmPt', 'V0A_DecayLength', 'V0A_Pos_isDuplicate', 'V0A_Neg_isDuplicate', 'V0A_Pos_Rec_Px', 'V0A_Pos_Rec_Py', 'V0A_Pos_Rec_Pz', 'V0A_Neg_Rec_Px', 'V0A_Neg_Rec_Py', 'V0A_Neg_Rec_Pz', 'V0A_Pos_NSigmaPion', 'V0A_Pos_NSigmaProton', 'V0A_Neg_NSigmaPion', 'V0A_Neg_NSigmaProton', 'V0A_Mother_PID', 'V0A_PID', 'V0A_Pos_PID', 'V0A_Neg_PID', 'V0B_Idx_Pos', 'V0B_Idx_Neg', 'V0B_Px', 'V0B_Py', 'V0B_Pz', 'V0B_X', 'V0B_Y', 'V0B_Z', 'V0B_Pos_Px', 'V0B_Pos_Py', 'V0B_Pos_Pz', 'V0B_Neg_Px', 'V0B_Neg_Py', 'V0B_Neg_Pz', 'V0B_isSignal', 'V0B_E_asK

In [3]:
#This function creates a list of keys, where V0A keys are switched with V0B keys
#It is used to switch V0A with V0B
def switch_keys(keys):
    switched_keys = []
    for key in keys:
        if "V0A" in key:
            switched_key = key.replace("V0A","V0B")
        else:
            switched_key = key.replace("V0B","V0A")
        
        switched_keys.append(switched_key)
    return switched_keys

#Function to read in the files, based on  and give out the data variables and their keys
#corresponding to given keys in a numpy array
#input: file directions, bool to randomize V0s to be either V0A and V0B for the whole file, list of keys
def read_in_files(files, randomizeAB,keys):
    keys_switcheroo = switch_keys(keys) #create switched key list for switching A and B

    data = [[] for i in range((len(keys)+1))] #empty 2D list for the data (+1 for candidate number not in the given keys) 
    for filename in files:
        try:
            with uproot.open(filename) as file:
                #calculate the candidate number
                Run_nbr = file['Sexaquarks']['RunNumber'].array()
                Dir_nbr = file['Sexaquarks']['DirNumber'].array()
                Dir_nbr = int(filename[-6])
                Event_nbr = file['Sexaquarks']['Event'].array()
                Idx_V0A = file['Sexaquarks']['Idx_V0A'].array()
                Idx_V0B = file['Sexaquarks']['Idx_V0B'].array()
                Candidate_nbr_file = Idx_V0B + 1000*Idx_V0A + 1000000*Event_nbr+1000000000*Dir_nbr +10000000000*Run_nbr           
                
                #append to data
                data[0].append(Candidate_nbr_file)
                
                #if randomizer is on and in 50% of the cases switch A and B
                if(randomizeAB & (random.random()>=0.5)):
                    for i,key in enumerate(keys_switcheroo):
                        data[i+1].append(file['Sexaquarks'][key].array())
                #else append variables is keys order
                else:
                    for i,key in enumerate(keys):
                        data[i+1].append(file['Sexaquarks'][key].array())
        except:
            print("File ",filename," not found")
    #result 3D list of variables containing 2D list of lists for each file
    #numpy concatenate all lists for files to get 2D numpy array of variables  
    for i in range(len(data)):
        data[i] = np.concatenate(data[i])
    #insert the key cand_nbr for the candidate numbers inserted in the data
    keys.insert(0,"Candidate_nbr")
    
    return  data,keys


def reshape_to_input(arrays):
    arrays_copy = arrays*1
    for i,array in enumerate(arrays):
        arrays_copy[i] = np.array([array]).T
       
    return arrays_copy

In [4]:
%%time
input_data = 0

with open('Data/Sexa_CustomV0.list') as f:
    input_data = f.read().splitlines()

randAB =True

#All keys one wants in their input Data excluding CandidateNBR, which gets computed on the fly. 
#Manually assigned!
keys = ["V0A_isSignal","V0B_isSignal","DCA","V0A_Pos_NSigmaPion", "V0A_Pos_NSigmaProton", "V0A_Neg_NSigmaPion", "V0A_Neg_NSigmaProton","V0B_Pos_NSigmaPion", "V0B_Pos_NSigmaProton", "V0B_Neg_NSigmaPion", "V0B_Neg_NSigmaProton",
        "V0A_ArmAlpha", "V0A_ArmPt", "V0B_ArmAlpha", "V0B_ArmPt", "V0A_X", "V0A_Y", "V0A_Z", "V0B_X", "V0B_Y", "V0B_Z", "X", "Y", "Z",
        "V0A_Px", "V0A_Py", "V0A_Pz", "V0B_Px", "V0B_Py", "V0B_Pz", "Px", "Py", "Pz", 
        'V0A_Pos_Px', 'V0A_Pos_Py', 'V0A_Pos_Pz', 'V0A_Neg_Px', 'V0A_Neg_Py', 'V0A_Neg_Pz', 'V0B_Pos_Px', 'V0B_Pos_Py', 'V0B_Pos_Pz', 'V0B_Neg_Px', 'V0B_Neg_Py', 'V0B_Neg_Pz',
        "V0A_Idx_Pos", "V0A_Idx_Neg", "V0B_Idx_Pos", "V0B_Idx_Neg", "V0A_Mother_PID", "V0A_PID", "V0A_Pos_PID", "V0A_Neg_PID", "V0B_Mother_PID", "V0B_PID", "V0B_Pos_PID", "V0B_Neg_PID","M"]

data,key_list = list(read_in_files(input_data,randAB,keys))


CPU times: user 1min 23s, sys: 1.3 s, total: 1min 24s
Wall time: 1min 46s


In [5]:
%%time
#This function expands the input data by user generated  variables
#expanded variables are:
#"V0A_Dist",V0B_Dist","Dist_V0A_Vert","Dist_V0B_Vert","Dist_Vert",
#"Opening_Angle","V0A_Transv_Mom","V0B_Transv_Mom","V0A_Cos_Point_Angle","V0B_Cos_Point_Angle","Opening_Angle_A","Opening_Angle_B"
def expand_input_data(data,key_list):
    #get indices for position and momentum in data using key_list
    idx_V0A_X = key_list.index("V0A_X")
    idx_V0B_X = key_list.index("V0B_X")
    idx_Vert_X = key_list.index("X")
    idx_V0A_Px = key_list.index("V0A_Px")
    idx_V0B_Px = key_list.index("V0B_Px")

    #define position and momentum vectors
    pos_A = np.array(data[idx_V0A_X:idx_V0A_X+3]).T
    pos_B = np.array(data[idx_V0B_X:idx_V0B_X+3]).T
    pos_vert = np.array(data[idx_Vert_X:idx_Vert_X+3]).T

    mom_A = np.array(data[idx_V0A_Px:idx_V0A_Px+3]).T
    mom_B = np.array(data[idx_V0B_Px:idx_V0B_Px+3]).T

    #calculate variables
    dist_A = np.linalg.norm(pos_A,axis=1)
    dist_B = np.linalg.norm(pos_B,axis=1)
    dist_vert = np.linalg.norm(pos_vert,axis=1)

    dist_A_vert = np.linalg.norm((pos_A-pos_vert),axis=1)
    dist_B_vert = np.linalg.norm((pos_B-pos_vert),axis=1)

    transv_dist_A = np.linalg.norm(pos_A[:,:2],axis=1)
    transv_dist_B = np.linalg.norm(pos_B[:,:2],axis=1)

    cos_point_angle_a = np.sum(pos_A*mom_A,axis=1) /(dist_A*np.linalg.norm(mom_A,axis=1))
    cos_point_angle_b = np.sum(pos_B*mom_B,axis=1) /(dist_B*np.linalg.norm(mom_B,axis=1))

    #insert in data and also insert in key list
    data.insert(16,dist_A)
    key_list.insert(16,"V0A_Dist")
    data.insert(17,dist_B)
    key_list.insert(17,"V0B_Dist")

    data.insert(18,dist_vert)
    key_list.insert(18,"Dist_Vert")
    data.insert(19,dist_B_vert)
    key_list.insert(19,"Dist_V0B_Vert")
    data.insert(19,dist_A_vert)
    key_list.insert(19,"Dist_V0A_Vert")

    #calculate transvers momenta and insert in data
    transv_mom_A = np.linalg.norm(mom_A[:,:2],axis=1)
    transv_mom_B = np.linalg.norm(mom_B[:,:2],axis=1)
    opening_angle = np.arccos(np.sum(mom_A*mom_B,axis=1)/(np.linalg.norm(mom_A,axis=1)*np.linalg.norm(mom_B,axis=1)))

    data.insert(21,opening_angle)
    data.insert(21,transv_mom_B)
    data.insert(21,transv_mom_A)
    key_list.insert(21,"Opening_Angle")
    key_list.insert(21,"V0B_Transv_Mom")
    key_list.insert(21,"V0A_Transv_Mom")

    data.insert(24,cos_point_angle_b)
    data.insert(24,cos_point_angle_a)
    key_list.insert(24,"V0B_Cos_Point_Angle")
    key_list.insert(24,"V0A_Cos_Point_Angle")


    #calculate opening angles of V0 A and B nad insert in data
    idx_A_pos_px = key_list.index("V0A_Pos_Px")
    idx_A_neg_px = key_list.index("V0A_Neg_Px")
    mom_A_pos = np.array(data[idx_A_pos_px:idx_A_pos_px+3]).T
    mom_A_neg = np.array(data[idx_A_neg_px:idx_A_neg_px+3]).T

    idx_B_pos_px = key_list.index("V0B_Pos_Px")
    idx_B_neg_px = key_list.index("V0B_Neg_Px")
    mom_B_pos = np.array(data[idx_B_pos_px:idx_B_pos_px+3]).T
    mom_B_neg = np.array(data[idx_B_neg_px:idx_B_neg_px+3]).T

    opening_angle_A = np.arccos(np.sum(mom_A_pos*mom_A_neg,axis=1)/(np.linalg.norm(mom_A_pos,axis=1)*np.linalg.norm(mom_A_neg,axis=1)))
    opening_angle_B = np.arccos(np.sum(mom_B_pos*mom_B_neg,axis=1)/(np.linalg.norm(mom_B_pos,axis=1)*np.linalg.norm(mom_B_neg,axis=1)))

    data.insert(idx_A_pos_px,opening_angle_B)
    data.insert(idx_A_pos_px,opening_angle_A)
    key_list.insert(idx_A_pos_px,"Opening_Angle_B")
    key_list.insert(idx_A_pos_px,"Opening_Angle_A")
    
    return data,key_list

print("Initial Number of parameters",len(data))
data,key_list = expand_input_data(data,key_list)
print("Final Number of parameters",len(data))
print("Total candidates: ",len(data[0]))
    
#make a numpy array out of the list of arrays of data
data = np.concatenate(reshape_to_input(data),axis=1)


Initial Number of parameters 59
Final Number of parameters 71
Total candidates:  3639689
CPU times: user 3.59 s, sys: 4.59 s, total: 8.18 s
Wall time: 8.28 s


In [6]:
#For the user to see the keys and indices and the cuts
key_to_index = {}
for i,key in enumerate(key_list):
    key_to_index[key] = i
print(key_to_index)

{'Candidate_nbr': 0, 'V0A_isSignal': 1, 'V0B_isSignal': 2, 'DCA': 3, 'V0A_Pos_NSigmaPion': 4, 'V0A_Pos_NSigmaProton': 5, 'V0A_Neg_NSigmaPion': 6, 'V0A_Neg_NSigmaProton': 7, 'V0B_Pos_NSigmaPion': 8, 'V0B_Pos_NSigmaProton': 9, 'V0B_Neg_NSigmaPion': 10, 'V0B_Neg_NSigmaProton': 11, 'V0A_ArmAlpha': 12, 'V0A_ArmPt': 13, 'V0B_ArmAlpha': 14, 'V0B_ArmPt': 15, 'V0A_Dist': 16, 'V0B_Dist': 17, 'Dist_Vert': 18, 'Dist_V0A_Vert': 19, 'Dist_V0B_Vert': 20, 'V0A_Transv_Mom': 21, 'V0B_Transv_Mom': 22, 'Opening_Angle': 23, 'V0A_Cos_Point_Angle': 24, 'V0B_Cos_Point_Angle': 25, 'V0A_X': 26, 'V0A_Y': 27, 'V0A_Z': 28, 'V0B_X': 29, 'V0B_Y': 30, 'V0B_Z': 31, 'X': 32, 'Y': 33, 'Z': 34, 'V0A_Px': 35, 'V0A_Py': 36, 'V0A_Pz': 37, 'V0B_Px': 38, 'V0B_Py': 39, 'V0B_Pz': 40, 'Px': 41, 'Py': 42, 'Pz': 43, 'Opening_Angle_A': 44, 'Opening_Angle_B': 45, 'V0A_Pos_Px': 46, 'V0A_Pos_Py': 47, 'V0A_Pos_Pz': 48, 'V0A_Neg_Px': 49, 'V0A_Neg_Py': 50, 'V0A_Neg_Pz': 51, 'V0B_Pos_Px': 52, 'V0B_Pos_Py': 53, 'V0B_Pos_Pz': 54, 'V0B_Neg_P

In [7]:
def signal_cut(arrays):
    return (arrays.T[1]*arrays.T[2]).astype(bool)

def true_bkg_cut(arrays):
    return ~(arrays.T[1]+arrays.T[2]).astype(bool)

def make_cut_greater_than_key(data,key_to_index,key,value):
    return data[data.T[key_to_index[key]] >= value]

def make_cut_smaller_than_key(data,key_to_index,key,value):
    return data[data.T[key_to_index[key]] <= value]

total_candidates = len(data)
total_signals    = len(data[signal_cut(data)])
total_background = len(data[true_bkg_cut(data)])


#make background reducing cuts

cut_data_A  = make_cut_greater_than_key(data,key_to_index,"V0A_Dist",52)
cut_data_AB = make_cut_greater_than_key(cut_data_A,key_to_index,"V0B_Dist",52)

distA_distB_cut_candidates = len(cut_data_AB)
distA_distB_cut_signals    = len(cut_data_AB[signal_cut(cut_data_AB)])
distA_distB_cut_background = len(cut_data_AB[true_bkg_cut(cut_data_AB)])



dca_cut_data = make_cut_smaller_than_key(cut_data_AB,key_to_index,"DCA", 1.1)


dcaAB_cut_candidates = len(dca_cut_data)
dcaAB_cut_signals    = len(dca_cut_data[signal_cut(dca_cut_data)])
dcaAB_cut_background = len(dca_cut_data[true_bkg_cut(dca_cut_data)])

dist_vert_cut_data = make_cut_greater_than_key(dca_cut_data,key_to_index,"Dist_Vert", 38)

dist_vert_cut_candidates = len(dist_vert_cut_data)
dist_vert_cut_signals    = len(dist_vert_cut_data[signal_cut(dist_vert_cut_data)])
dist_vert_cut_background = len(dist_vert_cut_data[true_bkg_cut(dist_vert_cut_data)])


print("                            total,  signal, background")
print("Total:                    ",total_candidates,total_signals,total_background)
print("DistA and DistB >= 52cm:  ",distA_distB_cut_candidates,distA_distB_cut_signals,distA_distB_cut_background)
print("DCA AB <= 1.1cm:          ",dcaAB_cut_candidates,dcaAB_cut_signals,dcaAB_cut_background)
print("Dist Vert >= 40cm:        ",dist_vert_cut_candidates,dist_vert_cut_signals,dist_vert_cut_background)



                            total,  signal, background
Total:                     3639689 4802 3511785
DistA and DistB >= 52cm:   3632183 4802 3504507
DCA AB <= 1.1cm:           231929 4742 223623
Dist Vert >= 40cm:         29702 4730 22828


In [ ]:
%%time
#save data and key list
np.save("DistCutData014.npy",dist_vert_cut_data)
np.save("Keys_DistCutData014.npy",key_list)


In [8]:
data = dist_vert_cut_data
#this function calculates the invariant mass based on assumptions, that the V0 is either a lambda or a kaon
def calc_inv_mass_under_assumptions(data):

    mass_proton = 0.938272
    mass_pion   = 0.139570
    
    #momentum vector
    idx_A_pos_px = key_list.index("V0A_Pos_Px")
    idx_A_neg_px = key_list.index("V0A_Neg_Px")

    idx_B_pos_px = key_list.index("V0B_Pos_Px")
    idx_B_neg_px = key_list.index("V0B_Neg_Px")


    V0A_Pos_Rec_P = data.T[idx_A_pos_px:idx_A_pos_px+3].T
    V0A_Neg_Rec_P = data.T[idx_A_neg_px:idx_A_neg_px+3].T
    V0B_Pos_Rec_P = data.T[idx_B_pos_px:idx_B_pos_px+3].T
    V0B_Neg_Rec_P = data.T[idx_B_neg_px:idx_B_neg_px+3].T
    
    V0A_Pos_Rec_P_tot = np.linalg.norm(V0A_Pos_Rec_P,axis=1)
    V0A_Neg_Rec_P_tot = np.linalg.norm(V0A_Neg_Rec_P,axis=1)
    V0B_Pos_Rec_P_tot = np.linalg.norm(V0B_Pos_Rec_P,axis=1)
    V0B_Neg_Rec_P_tot = np.linalg.norm(V0B_Neg_Rec_P,axis=1)
 
    #momentum
    V0A_Pos_Rec_P_tot = np.linalg.norm(V0A_Pos_Rec_P,axis=1)
    V0A_Neg_Rec_P_tot = np.linalg.norm(V0A_Neg_Rec_P,axis=1)
    V0B_Pos_Rec_P_tot = np.linalg.norm(V0B_Pos_Rec_P,axis=1)
    V0B_Neg_Rec_P_tot = np.linalg.norm(V0B_Neg_Rec_P,axis=1) 

    #energy as pion or proton E^2 = m^2 + p^2
    V0A_Pos_Rec_E_pion    = (mass_pion**2     + V0A_Pos_Rec_P_tot**2)**0.5
    V0A_Neg_Rec_E_pion    = (mass_pion**2     + V0A_Neg_Rec_P_tot**2)**0.5
    V0A_Neg_Rec_E_prot    = (mass_proton**2   + V0A_Neg_Rec_P_tot**2)**0.5

    V0B_Pos_Rec_E_pion    = (mass_pion**2     + V0B_Pos_Rec_P_tot**2)**0.5
    V0B_Neg_Rec_E_pion    = (mass_pion**2     + V0B_Neg_Rec_P_tot**2)**0.5
    V0B_Neg_Rec_E_prot    = (mass_proton**2   + V0B_Neg_Rec_P_tot**2)**0.5

    #4 momenta P=(E,px,py,pz)
    V0A_Pos_Rec_4mom_pion = np.array((V0A_Pos_Rec_E_pion,V0A_Pos_Rec_P[:,0],V0A_Pos_Rec_P[:,1],V0A_Pos_Rec_P[:,2]))
    V0A_Neg_Rec_4mom_pion = np.array((V0A_Neg_Rec_E_pion,V0A_Neg_Rec_P[:,0],V0A_Neg_Rec_P[:,1],V0A_Neg_Rec_P[:,2]))
    V0A_Neg_Rec_4mom_prot = np.array((V0A_Neg_Rec_E_prot,V0A_Neg_Rec_P[:,0],V0A_Neg_Rec_P[:,1],V0A_Neg_Rec_P[:,2]))

    V0B_Pos_Rec_4mom_pion = np.array((V0B_Pos_Rec_E_pion,V0B_Pos_Rec_P[:,0],V0B_Pos_Rec_P[:,1],V0B_Pos_Rec_P[:,2]))
    V0B_Neg_Rec_4mom_pion = np.array((V0B_Neg_Rec_E_pion,V0B_Neg_Rec_P[:,0],V0B_Neg_Rec_P[:,1],V0B_Neg_Rec_P[:,2]))
    V0B_Neg_Rec_4mom_prot = np.array((V0B_Neg_Rec_E_prot,V0B_Neg_Rec_P[:,0],V0B_Neg_Rec_P[:,1],V0B_Neg_Rec_P[:,2]))

    #4 momenta lambda /K0
    V0A_Rec_4mom_Lamb     = V0A_Pos_Rec_4mom_pion + V0A_Neg_Rec_4mom_prot
    V0A_Rec_4mom_K0sh     = V0A_Pos_Rec_4mom_pion + V0A_Neg_Rec_4mom_pion
    V0B_Rec_4mom_Lamb     = V0B_Pos_Rec_4mom_pion + V0B_Neg_Rec_4mom_prot
    V0B_Rec_4mom_K0sh     = V0B_Pos_Rec_4mom_pion + V0B_Neg_Rec_4mom_pion

    #invyarant mass squared: M^2 = E^2 - p^2
    V0A_Rec_invM_Lamb_2   = V0A_Rec_4mom_Lamb[0]**2 - np.sum(V0A_Rec_4mom_Lamb[1:]**2,axis=0)
    V0A_Rec_invM_K0sh_2   = V0A_Rec_4mom_K0sh[0]**2 - np.sum(V0A_Rec_4mom_K0sh[1:]**2,axis=0)
    V0B_Rec_invM_Lamb_2   = V0B_Rec_4mom_Lamb[0]**2 - np.sum(V0B_Rec_4mom_Lamb[1:]**2,axis=0)
    V0B_Rec_invM_K0sh_2   = V0B_Rec_4mom_K0sh[0]**2 - np.sum(V0B_Rec_4mom_K0sh[1:]**2,axis=0)
    
    return V0A_Rec_invM_Lamb_2,V0A_Rec_invM_K0sh_2,V0B_Rec_invM_Lamb_2,V0B_Rec_invM_K0sh_2

V0A_Rec_invM_Lamb_2,V0A_Rec_invM_K0sh_2,V0B_Rec_invM_Lamb_2,V0B_Rec_invM_K0sh_2 = calc_inv_mass_under_assumptions(data)



In [9]:
#calculate a switch map to swap A and B 
key_list_switch=switch_keys(key_list)
key_switch_map = [key_list_switch.index(keyy)for keyy in key_list ]

#swap A and B if the reconstructed invariant mass of A under assumption Lambda is bigger than 
#inv mass B under assumption Lambda
for i in range(len(data)):
    if ((V0A_Rec_invM_Lamb_2[i]-V0B_Rec_invM_Lamb_2[i]) >0 ):
        data[i] = data[i][key_switch_map]

#calculate inv masses again        
V0A_Rec_invM_Lamb_2,V0A_Rec_invM_K0sh_2,V0B_Rec_invM_Lamb_2,V0B_Rec_invM_K0sh_2 = calc_inv_mass_under_assumptions(data)
    

In [10]:
#number total candidates
total_candidates = len(data)
total_signals    = len(data[signal_cut(data)])
total_background = len(data[true_bkg_cut(data)])

#cut on inv mass of A assumed as Lambda
cut_data_A = data[V0A_Rec_invM_Lamb_2 <= 1.35]
V0B_Rec_invM_K0sh_2_mask = V0B_Rec_invM_K0sh_2[V0A_Rec_invM_Lamb_2 <= 1.35]

InvMA_cut_candidates = len(cut_data_A)
InvMA_cut_signals    = len(cut_data_A[signal_cut(cut_data_A)])
InvMA_cut_background = len(cut_data_A[true_bkg_cut(cut_data_A)])

#cut on inv mass of B assumed as Kaon
cut_data_both = cut_data_A[V0B_Rec_invM_K0sh_2_mask <= 0.35]
V0B_Rec_invM_K0sh_2_mask = V0B_Rec_invM_K0sh_2_mask[V0B_Rec_invM_K0sh_2_mask <= 0.35]

InvMAB_cut_candidates = len(cut_data_both)
InvMAB_cut_signals    = len(cut_data_both[signal_cut(cut_data_both)])
InvMAB_cut_background = len(cut_data_both[true_bkg_cut(cut_data_both)])


print("                            total,  signal, background")
print("Total:                     ",total_candidates,total_signals,total_background)
print("Inv M^2 A _Lambda <= 1.35: ",InvMA_cut_candidates,InvMA_cut_signals,InvMA_cut_background)
print("Inv M^2 A _Lambda <= 0.35: ",InvMAB_cut_candidates,InvMAB_cut_signals,InvMAB_cut_background)



                            total,  signal, background
Total:                      29702 4730 22828
Inv M^2 A _Lambda <= 1.35:  14461 4661 8283
Inv M^2 A _Lambda <= 0.35:  8786 4571 3370


In [12]:
V0A_Rec_invM_Lamb_2,V0A_Rec_invM_K0sh_2,V0B_Rec_invM_Lamb_2,V0B_Rec_invM_K0sh_2 = calc_inv_mass_under_assumptions(cut_data_both)

#put inv_mass squared as variable into the dataset
data3 = np.concatenate((cut_data_both.T[:24],[V0B_Rec_invM_K0sh_2],cut_data_both.T[24:]))
data3 = np.concatenate((data3[:24],[V0B_Rec_invM_Lamb_2],data3[24:]))
data3 = np.concatenate((data3[:24],[V0A_Rec_invM_K0sh_2],data3[24:]))
data3 = np.concatenate((data3[:24],[V0A_Rec_invM_Lamb_2],data3[24:]))
key_list.insert(24,"V0B_Rec_invM_K0sh_2")
key_list.insert(24,"V0B_Rec_invM_Lamb_2")
key_list.insert(24,"V0A_Rec_invM_K0sh_2")
key_list.insert(24,"V0A_Rec_invM_Lamb_2")


key_index_pair={}
for i,key in enumerate(key_list):
    key_index_pair[i]=key
print(key_index_pair)

{0: 'Candidate_nbr', 1: 'V0A_isSignal', 2: 'V0B_isSignal', 3: 'DCA', 4: 'V0A_Pos_NSigmaPion', 5: 'V0A_Pos_NSigmaProton', 6: 'V0A_Neg_NSigmaPion', 7: 'V0A_Neg_NSigmaProton', 8: 'V0B_Pos_NSigmaPion', 9: 'V0B_Pos_NSigmaProton', 10: 'V0B_Neg_NSigmaPion', 11: 'V0B_Neg_NSigmaProton', 12: 'V0A_ArmAlpha', 13: 'V0A_ArmPt', 14: 'V0B_ArmAlpha', 15: 'V0B_ArmPt', 16: 'V0A_Dist', 17: 'V0B_Dist', 18: 'Dist_Vert', 19: 'Dist_V0A_Vert', 20: 'Dist_V0B_Vert', 21: 'V0A_Transv_Mom', 22: 'V0B_Transv_Mom', 23: 'Opening_Angle', 24: 'V0A_Rec_invM_Lamb_2', 25: 'V0A_Rec_invM_K0sh_2', 26: 'V0B_Rec_invM_Lamb_2', 27: 'V0B_Rec_invM_K0sh_2', 28: 'V0A_Cos_Point_Angle', 29: 'V0B_Cos_Point_Angle', 30: 'V0A_X', 31: 'V0A_Y', 32: 'V0A_Z', 33: 'V0B_X', 34: 'V0B_Y', 35: 'V0B_Z', 36: 'X', 37: 'Y', 38: 'Z', 39: 'V0A_Px', 40: 'V0A_Py', 41: 'V0A_Pz', 42: 'V0B_Px', 43: 'V0B_Py', 44: 'V0B_Pz', 45: 'Px', 46: 'Py', 47: 'Pz', 48: 'Opening_Angle_A', 49: 'Opening_Angle_B', 50: 'V0A_Pos_Px', 51: 'V0A_Pos_Py', 52: 'V0A_Pos_Pz', 53: 'V0A_N

In [15]:
%%time
#save data and key list
np.save("MassCutData016.npy",data3.T)
np.save("Keys_MassCutData016.npy",key_list)

CPU times: user 4.9 ms, sys: 4.72 ms, total: 9.63 ms
Wall time: 1.44 s
